In [1]:
# ================================================================
# Bachelor Thesis — Fairness in Toxic Comment Classification
# ---------------------------------------------------------------
# Notebook: base_model_training_and_evaluation.ipynb
# Author: Philipp Stocker
# Created: 14.11.2025
# Purpose: TODO: fill purpose
# ================================================================

# --- Basic setup ---
import os
import sys
import warnings
warnings.filterwarnings("ignore")  # keep output clean for reports

# Automatically add project root to path so src/ modules are importable
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

# --- Standard imports ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import joblib

from src.modeling import train_lr_tfidf
from src.modeling import predict_with_model
from src.modeling import extract_feature_and_label_arrays

# --- "Global variables" ---
DATA_RAW = os.path.join(project_root, "data", "raw")
DATA_PROCESSED = os.path.join(project_root, "data", "processed")

IDENTITY_COLUMNS = ["male", "female", "heterosexual", "homosexual_gay_or_lesbian", "bisexual", "transgender", "other_gender", "other_sexual_orientation"]

print("✅ Environment ready")


✅ Environment ready


Load feature and label arrays

In [2]:
x_train, y_train = extract_feature_and_label_arrays(joblib.load(os.path.join(DATA_PROCESSED, "train_tfidf_bundle.joblib")))
x_val, y_val = extract_feature_and_label_arrays(joblib.load(os.path.join(DATA_PROCESSED, "val_tfidf_bundle.joblib")))
x_test, y_test = extract_feature_and_label_arrays(joblib.load(os.path.join(DATA_PROCESSED, "test_tfidf_bundle.joblib")))

BASE MODEL TRAINING

Train base model with according parameters

In [ ]:
# Train base model: Logistic Regression with TF-IDF features (per default with calibration)
base_models = train_lr_tfidf(
    x_train=x_train,
    y_train=y_train,
    x_val=x_val,
    y_val=y_val
)

# Save trained base models
joblib.dump(base_models, project_root + "/models/tfidf_base_models.joblib")

BASE MODEL EVALUATION

Load base models for further evaluation

In [3]:
base_models = joblib.load(project_root + "/models/tfidf_base_models.joblib")

Conduct prediction on test dataset with base models

In [ ]:
y_test_proba, y_test_pred = predict_with_model(base_models["model"], x_test, threshold=0.5)

# Predict with uncalibrated model (if it exists) for comparison
if base_models["uncalibrated_model"] is not None:
    y_test_proba_uncalibrated, y_test_pred_uncalibrated = predict_with_model(base_models["uncalibrated_model"], x_test, threshold=0.5)

Compare number of toxic to non-toxic comments (across both the model with and without calibration)

In [7]:
count_toxic = np.sum(y_test_pred)
count_nontoxic = len(y_test_pred) - count_toxic
print(f"Number of toxic comments: {count_toxic}")
print(f"Number of non-toxic comments: {count_nontoxic}")

if base_models["uncalibrated_model"] is not None:
    count_toxic_uncalibrated = np.sum(y_test_pred_uncalibrated)
    count_nontoxic_uncalibrated = len(y_test_pred_uncalibrated) - count_toxic_uncalibrated
    print(f"Number of toxic comments (uncalibrated): {count_toxic_uncalibrated}")
    print(f"Number of non-toxic comments (uncalibrated): {count_nontoxic_uncalibrated}")

Number of toxic comments: 13540
Number of non-toxic comments: 257191
Number of toxic comments (uncalibrated): 42944
Number of non-toxic comments (uncalibrated): 227787
